<a href="https://colab.research.google.com/github/nneko1231/RoopColab/blob/main/DeepFake_V2.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Drive Mount

In [ ]:
# will mount drive if MOUNT_DRIVE is True
import os
import subprocess
import shutil
from google.colab import drive, output

# Add a parameter to control mounting Google Drive
MOUNT_DRIVE = True  #@param {type:"boolean"}

# mount drive if MOUNT_DRIVE is True
if MOUNT_DRIVE:
    drive.mount('/content/drive')

# 2. Installing

In [ ]:
# Select Execution Environment
Processing_Use = "GPU"  # @param ["GPU", "CPU"]

# Clone the repository
!git clone https://github.com/nneko1231/roop
%cd roop

# Print the selected execution environment
if Processing_Use == "GPU":
    print("You have selected GPU. Installing GPU-specific libraries...")
    # Install GPU-specific libraries like onnxruntime-gpu
    !pip install onnxruntime-gpu
else:
    print("You have selected CPU. Installing CPU-specific libraries...")
    # Install CPU-specific libraries like onnxruntime
    !pip install onnxruntime

# Install other required dependencies
!pip install -r requirements.txt

# Modify a specific file (make sure this change is required)
file_path = '/usr/local/lib/python3.11/dist-packages/jax/_src/dtypes.py'

with open(file_path, 'r') as f:
    file_contents = f.read()

file_contents = file_contents.replace('np.dtypes', 'np.dtype')

with open(file_path, 'w') as f:
    f.write(file_contents)

In [ ]:
#@title Model Download
!wget https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O inswapper_128.onnx
!mkdir models
!mv inswapper_128.onnx ./models

# 3. Settings

In [14]:
SOURCE_PATH = "SOURCE.jpg" #@param {type:"string"}
TARGET_PATH = "TARGET.mp4" #@param {type:"string"}
OUTPUT_PATH = "OUTPUT.mp4" #@param {type:"string"}
#@markdown - Recommended to use face_swapper if only using CPU.
frame_processor = "face_swapper face_enhancer" # @param ["face_swapper face_enhancer","face_swapper","face_enhancer"]
#@markdown - Recommended GPU 8, CPU 2
execution_threads = 8 #@param {type:"slider", min:1, max:8, step:1}
keep_fps = True  #@param {type:"boolean"}
keep_frames = True  #@param {type:"boolean"}
skip_audio = False  #@param {type:"boolean"}
temp_frame_format = "png" #@param ["jpg" , "png"]
#@markdown - Recommended 80–95 to balance quality and efficiency.
temp_frame_quality = 80 # @param {"type":"slider","min":0,"max":100,"step":1}
output_video_encoder = "h264_nvenc" # @param ["libx264","libx265","libvpx-vp9","h264_nvenc","hevc_nvenc"]
#@markdown - For professional or publication results, a value of 80+ is recommended.
#@markdown - For a quick test or preview, a value of 40–60 can be used.
output_video_quality = 80 # @param {"type":"slider","min":0,"max":100,"step":1}
max_memory = 12 # @param {"type":"slider","min":0.1,"max":12,"step":0.1}


# 4. Run


In [ ]:
#@title Run Roop
import os

def build_backend_command() -> str:
    # Start building the command with basic parameters
    command = (
        'python3 /content/roop/run.py' +
        f' -s "{SOURCE_PATH}" ' +
        f' -t "{TARGET_PATH}" ' +
        f' -o "{OUTPUT_PATH}" ' +
        f' --execution-threads {execution_threads}'  # Default number of execution threads
    )

    # List of encoders that do not require CUDA
    non_cuda_encoders = ["libx264", "libx265", "libvpx-vp9"]

    # If the selected encoder is one of the non-CUDA encoders
    if output_video_encoder in non_cuda_encoders:
        # Disable CUDA and run with CPU
        command += ' --execution-provider cpu '
    else:
        # Use CUDA for encoding (for encoders like h264_nvenc, hevc_nvenc)
        command += ' --execution-provider cuda '

    # Always add the selected frame processors
    command += f' --frame-processor {frame_processor}'

    # Add options if they are set to True
    if keep_fps:
        command += ' --keep-fps '
    if keep_frames:
        command += ' --keep-frames '
    if skip_audio:
        command += ' --skip-audio '

    # Add the other parameters for frame format, video encoder, etc.
    command += (
        f' --temp-frame-format {temp_frame_format} ' +
        f' --temp-frame-quality {temp_frame_quality} ' +
        f' --output-video-encoder {output_video_encoder} ' +
        f' --output-video-quality {output_video_quality} ' +
        f' --max-memory {max_memory} '
    )
    return command

try:
    # Write the command to a shell script file
    with open("command.sh", "w") as f:
        f.write(build_backend_command())

    # Execute the command from the shell script
    !bash command.sh

except KeyboardInterrupt:
    print("The process was terminated by the user.")

finally:
    # Ensure the command.sh file is deleted after execution
    if os.path.exists("command.sh"):
        os.remove("command.sh")
        print("The command.sh file is deleted.")